<center>
<h1 style="background-color:orange; color:white" >Stroke Prediction EDA-ML</h1>

<center><img
src="https://www.mmcoakridge.com/wp-content/uploads/sites/14/2020/07/Brain-Attack-Stroke-iStock-1168179084.jpg" style="width:90%;height:40%;">
</center>
    
<br>    
    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:orange; color:white" data-
    toggle="list"  role="tab" aria-controls="home">Content</h3>  
<ul>
        <li>EDA</li>
        <li>Modelling</li>
    </ul>
  

In [ ]:
#Öznitelik Bilgileri
# kimlik: benzersiz tanımlayıcı
# cinsiyet: "Erkek", "Kadın" veya "Diğer"
# yaş: hastanın yaşı
# hipertansiyon: Hastada hipertansiyon yoksa 0, hipertansiyon varsa 1
# heart_disease: Hastada herhangi bir kalp hastalığı yoksa 0, kalp hastalığı varsa 1
# ever_married: "Hayır" veya "Evet"
# work_type: "çocuklar", "Govt_jov", "Never_worked", "Özel" veya "Serbest Meslek Sahibi"
# Residence_type: "Kırsal" veya "Kentsel"
# avg_glucose_level: kandaki ortalama glikoz seviyesi
# bmi: vücut kitle indeksi
# smoking_status: "önceden içilmiş", "hiç içmemiş", "sigara içiyor" veya "Bilinmiyor" *
# inme: Hasta inme geçirdiyse 1 veya yoksa 0
# Not: Smoking_status'taki "Unknown", bilgilerin bu hasta için mevcut olmadığı anlamına gelir

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score
from sklearn import metrics
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 170)
warnings.filterwarnings('ignore')

In [ ]:
##############################################################
# 1. Data Preperation
##############################################################

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.nunique() #herbir değişken için eşsiz değer sayılarımız

In [ ]:
df.isnull().any().sum()  #Eksik değer var mı hiç?

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().T

In [ ]:

def grab_col_names(dataframe, cat_th=10, car_th=20):

    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]

    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]

    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]

    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    return cat_cols, cat_but_car, num_cols, num_but_cat

#veri seti gözlemler hakkında inceleme yapıldı
cat_cols, cat_but_car, num_cols, num_but_cat = grab_col_names(df)

In [ ]:
def data_understand(df):
    print("DF SHAPE:", df.shape)
    print("------------------------------------------------------------------------")
    print("OUTCOME 1 DF RATIO:", len(df[df["stroke"] == 1]) / len(df))
    print("OUTCOME 0 DF RATIO:", len(df[df["stroke"] == 0]) / len(df))
    print("------------------------------------TYPES------------------------------------")
    print(df.dtypes)
    print("------------------------------------HEAD------------------------------------")
    print(df.head())
    print("-------------------------------------TAİL-----------------------------------")
    print(df.tail())
    print("------------------------------------DESCRİBE------------------------------------")
    print(df.describe().T)
    print("-------------------------------------QUANTİLE-----------------------------------")
    print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
    print("-----------------------------------CORR-------------------------------------")
    # Isı haritasında, daha parlak renkler daha fazla korelasyonu gösterir.

    corr = df.corr()
    print(corr)
    print("--------------------------------------HEATMAP----------------------------------")
    sns.heatmap(corr,
                xticklabels=corr.columns,
                yticklabels=corr.columns)
    plt.show()


data_understand(df)

In [ ]:
#Veri setindeki eksik değerleri sorgulamak için
def df_questioning_null(df):

    print(f"Veri kümesinde hiç boş değer var mı?: {df.isnull().values.any()}")
    if df.isnull().values.any():
        null_values = df.isnull().sum()
        print(f"Hangi sütunlarda eksik değerler var?:\n{null_values[null_values > 0]}")

df_questioning_null(df)

In [ ]:

#Aykırı değerleri yakalamak için
def outlier_thresholds(dataframe, col_name):
    quartile1 = dataframe[col_name].quantile(0.25)
    quartile3 = dataframe[col_name].quantile(0.75)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


In [ ]:
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True #return ün işlenebilir bir nesne olması lazım
    else:
        return False


In [ ]:
check_outlier(df, "age")  #False

In [ ]:
check_outlier(df,"avg_glucose_level")  #True

In [ ]:
df_numerical=df[['age','avg_glucose_level','bmi']]
df_categorical=df[['gender', 'hypertension', 'heart_disease', 'ever_married','work_type', 'Residence_type',
                       'smoking_status', 'stroke']]

In [ ]:
fig=plt.figure(figsize=(20,15),dpi=100)
sns.pairplot(data=df,hue='stroke',size=2,palette='OrRd')
plt.show()


In [ ]:

# # Marrital Durumunu, İkametgahını ve Cinsiyetini 0'lara ve 1'lere dönüştürüldü
df['gender']=df['gender'].apply(lambda x : 1 if x=='Male' else 0)
df["Residence_type"] = df["Residence_type"].apply(lambda x: 1 if x=="Urban" else 0)
df["ever_married"] = df["ever_married"].apply(lambda x: 1 if x=="Yes" else 0)
# # Sigara türü bilinmeyen gözlemlerin kaldırıldı
df=df[df['smoking_status']!='Unknown']
bmi_indexes = df[df["bmi"].isna()].index
df.drop(bmi_indexes, errors="ignore", inplace=True)

df.head()

In [ ]:
def replace_with_thresholds(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if low_limit > 0: #zaten - çıkan bir limit baskılanamaz
        dataframe.loc[(dataframe[col_name] < low_limit), col_name] = low_limit
        dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit
    else:
        dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit

for col in num_cols:
    replace_with_thresholds(df, col)

df = pd.get_dummies(df, drop_first=True)

In [ ]:
#Accuracy Skoru bulmak için
def model_comparison_accuracy_score(model_name, model_object, data, target_col):
    np.random.seed(0)
    y = data[[target_col]]
    X = data.drop(target_col, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
    model = model_object()
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    print(f"-{model_name}-".center(50, "#"))
    print(confusion_matrix(y_pred, y_test))
    return accuracy_score(y_pred, y_test)

In [ ]:
#Sensitivity Skoru bulmak için

def model_comparison_sensitivity_score(model_name, model_object, data, target_col):
    np.random.seed(0)
    y = data[[target_col]]
    X = data.drop(target_col, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
    model = model_object()
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    cm1 = confusion_matrix(y_pred, y_test)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    return sensitivity1

In [ ]:
#Specificity Skoru bulmak için
def model_comparison_specificity_score(model_name, model_object, data, target_col):
    np.random.seed(0)
    y = data[[target_col]]
    X = data.drop(target_col, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
    model = model_object()
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    cm1 = confusion_matrix(y_pred, y_test)
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    return specificity1


In [ ]:
model_dict = {"Karar Ağaçları": DecisionTreeClassifier,
              "En Yakın Komşu": KNeighborsClassifier,
              "Bayes Sınıflandırma": GaussianNB,
              "Yapay Sinir Ağları": MLPClassifier,
              "Destek Vektör Makineleri": SVC}

In [ ]:
score_dict = {}
for name, model in model_dict.items():
    score = model_comparison_accuracy_score(name, model, df, "stroke")
    score_dict[name] = round(score, 4)

score_dict_2 = {}
for name, model in model_dict.items():
    score = model_comparison_sensitivity_score(name, model, df, "stroke")
    score_dict_2[name] = round(score, 4)


score_dict_3= {}
for name, model in model_dict.items():
    score = model_comparison_specificity_score(name, model, df, "stroke")
    score_dict_3[name] = round(score, 4)

In [ ]:
print(" ACCURACY SCORES ".center(50, "#"))
print(score_dict)

print(" SENSITIVITY SCORES ".center(50, "#"))
print(score_dict_2)

print(" SPESIFYCIY SCORES ".center(50, "#"))
print(score_dict_3)
